In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
import bw_processing as bwp
from fs.zipfs import ZipFS

from tqdm import tqdm

import numpy as np
import os
import sys
import time

In [2]:
%%time
bi.restore_project_directory("/Users/ajakobs/Documents/data/hybrid-eco-exio/CH_household_hybrid_project_eco391_ex381_march_28.tar.gz")


Restoring project backup archive - this could take a few minutes...
CPU times: user 8.15 s, sys: 2.21 s, total: 10.4 s
Wall time: 11 s


'eco-ex-March-2023'

In [3]:
bd.projects

Brightway2 projects manager with 8 objects:
	AB_test_exploring_EI39
	CCL dev
	default
	eco-ex
	eco-ex-March-2023
	ecoinvent 3.3 cutoff
	validation
	validation_ei39
Use `projects.report()` to get a report on all projects.

In [4]:
bd.projects.set_current("eco-ex-March-2023")

In [5]:
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff
	exiobase-3.8.1-monetary
	exiobase-3.8.1-monetary biosphere
	swiss consumption 1.0

In [6]:
co = bd.Database("swiss consumption 1.0")

In [7]:
def calculate_iterations_tofile(acts, tot_len, data_objs, method, save_dir, use_samples=True, iterations=25, uncertain_contributions=False, bar=True):
    """calculates inventories for all activities for a given number of iterations and saves the
    inventories as an array with size (n_biosphereflows, n_activites) as npy files in the specified dir
    If uncertain_contributions==True, it will calculate the input (product categories) contributions for the last activity
    in the activity list 'acts' if this is ch hh average consumption disaggregated, years 151617, and otherwise raise an 
    assertion error.
    impacts for each run"""
    lca = bc.LCA({acts[0]: 1}, method=method, use_distributions=True, use_arrays=use_samples)
    lca.lci()
    lca.lcia()
    
    #make results array
    results_array = np.zeros((lca.inventory.shape[0],tot_len))
    results_array[:,0] = lca.inventory.sum(axis=1).A[:,0]
    
    for i,act in enumerate(acts[1:]):
        lca.lcia({act.id: 1})
        results_array[:,i+1] = lca.inventory.sum(axis=1).A[:,0]
    if uncertain_contributions:
        assert act['name'] == 'ch hh average consumption disaggregated, years 151617'
        column = lca.dicts.activity[act.id]
        for k,exc in enumerate(sorted(act.technosphere(), key=lambda x: x['name'])):
            amount = -lca.technosphere_matrix[lca.dicts.activity[exc.input.id],column]
            lca.lcia({exc.input.id: amount})
            results_array[:,len(acts)+k] = lca.inventory.sum(axis=1).A[:,0]
    np.save(os.path.join(save_dir,f'hh_consumption_inventory_{0}.npy'), results_array)
    print('saved to',os.path.join(save_dir,f'hh_consumption_inventory_{0}.npy'))
    
    if bar:
        for i in tqdm(range(iterations - 1)):
            next(lca)
            for j,act in enumerate(acts):
                lca.lcia({act.id: 1})
                results_array[:,j] = lca.inventory.sum(axis=1).A[:,0]
            if uncertain_contributions:
                assert act['name'] == 'ch hh average consumption disaggregated, years 151617'
                column = lca.dicts.activity[act.id]
                for k,exc in enumerate(sorted(act.technosphere(), key=lambda x: x['name'])):
                    amount = -lca.technosphere_matrix[lca.dicts.activity[exc.input.id],column]
                    lca.lcia({exc.input.id: amount})
                    results_array[:,j+k+1] = lca.inventory.sum(axis=1).A[:,0]
            np.save(os.path.join(save_dir,f'hh_consumption_inventory_{i+1}.npy'),results_array)
    else:
        for i in range(iterations - 1):
            next(lca)
            for j,act in enumerate(acts):
                lca.lcia({act.id: 1})
                results_array[:,j] = lca.inventory.sum(axis=1).A[:,0]
            if uncertain_contributions:
                assert act['name'] == 'ch hh average consumption disaggregated, years 151617'
                column = lca.dicts.activity[act.id]
                for k,exc in enumerate(sorted(act.technosphere(), key=lambda x: x['name'])):
                    amount = -lca.technosphere_matrix[lca.dicts.activity[exc.input.id],column]
                    lca.lcia({exc.input.id: amount})
                    results_array[:,j+k+1] = lca.inventory.sum(axis=1).A[:,0]
            np.save(os.path.join(save_dir,f'hh_consumption_inventory_{i+1}.npy'),results_array)
    return lca

## define act list

In [8]:
ipcc = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
ipcc

('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [9]:
# redo all to test results
fu_list = [act for act in sorted(co, key=lambda x: x['name'])]
len(fu_list)

314

In [10]:
# prepare datapackages
func_unit, data_objs, _ = bd.prepare_lca_inputs(demand={fu_list[0]: 1}, method=ipcc)
# now add the datapackage cpntaining the household cluster samples
data_objs.append(bwp.load_datapackage(ZipFS("./write_files/ccl_dev/household_archetypes.zip")))

In [11]:
%%time
# calculate mc runs using direct sampling from datapackage
n_iterations = 1000
total_len = len(fu_list)
lca = calculate_iterations_tofile(fu_list, tot_len=total_len, data_objs=data_objs, method=ipcc,
                                  save_dir='/Users/ajakobs/Documents/CCL/results/MC_results_directly_sampled/',
                                  use_samples=True, iterations=n_iterations, uncertain_contributions=False, bar=True)


saved to /Users/ajakobs/Documents/CCL/results/MC_results_directly_sampled/hh_consumption_inventory_0.npy


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 999/999 [1:37:49<00:00,  5.88s/it]

CPU times: user 9h 3min 33s, sys: 17min 28s, total: 9h 21min 1s
Wall time: 1h 37min 56s


In [12]:
# write order of activities to file
fu_list_names = [act['name'] for act in fu_list]
with open('/Users/ajakobs/Documents/CCL/results/MC_results_directly_sampled/activity_names.txt','w') as fh:
    fh.write('\n'.join(fu_list_names))

In [19]:
%%time
n_iterations = 1000
total_len = len(fu_list)
lca = calculate_iterations_tofile(fu_list, tot_len=total_len, data_objs=data_objs, method=ipcc,
                                  save_dir='/Users/ajakobs/Documents/CCL/results/MC_results/',
                                  use_samples=False, iterations=n_iterations, uncertain_contributions=False, bar=True)


saved to /Users/ajakobs/Documents/CCL/results/MC_results/hh_consumption_inventory_0.npy


100%|███████████████████████████████████████████████████████████████████████████████████| 999/999 [1:32:10<00:00,  5.54s/it]

CPU times: user 8h 37min 31s, sys: 15min 26s, total: 8h 52min 57s
Wall time: 1h 32min 18s


In [21]:
# write order of activities to file
fu_list_names = [act['name'] for act in fu_list]
with open('/Users/ajakobs/Documents/CCL/results/MC_results/activity_names.txt','w') as fh:
    fh.write('\n'.join(fu_list_names))

In [17]:
fu_list

['Accommodation services' (CHF, CH, None),
 'Air travels, tickets' (CHF, CH, None),
 'Alcoholic and non-alcoholic beer' (litre, CH, None),
 'Alcoholic and non-alcoholic liqueurs and liqueur-based aperitifs' (litre, CH, None),
 'Apples' (kilogram, CH, None),
 'Bakery products (sweet and salty)' (kilogram, CH, None),
 'Banana' (kilogram, CH, None),
 'Beans and peas' (kilogram, CH, None),
 'Beef' (kilogram, CH, None),
 'Beets and other root vegetables' (kilogram, CH, None),
 'Berries' (kilogram, CH, None),
 'Bicycles' (person kilometer, CH, None),
 'Books and brochures' (CHF, CH, None),
 'Brassicas' (kilogram, CH, None),
 'Bread' (kilogram, CH, None),
 'Bus, tickets and travelcards' (person kilometer, CH, None),
 'Butter' (kilogram, CH, None),
 'Canteens' (CHF, CH, None),
 'Central heating or district heating of principal residence' (megajoule, CH, None),
 'Chocolate' (kilogram, CH, None),
 'Cinema' (CHF, CH, None),
 'Cleaning articles' (CHF, CH, None),
 'Cleaning, repair and hire of clot